In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Define updated classes and functions
class Time_Series_Dataset(Dataset):
    def __init__(self, inputs, decoder_inputs, outputs):
        self.inputs = inputs
        self.decoder_inputs = decoder_inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = self.inputs[idx]
        decoder_input = self.decoder_inputs[idx]
        y = self.outputs[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(decoder_input, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        out = self.fc(out)
        return out, hidden, cell

class EncoderDecoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(EncoderDecoderLSTM, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers)
        self.decoder = Decoder(1, hidden_size, output_size, num_layers)

    def forward(self, encoder_inputs, decoder_inputs):
        hidden, cell = self.encoder(encoder_inputs)
        decoder_inputs = decoder_inputs.unsqueeze(-1)  # Add feature dimension to decoder inputs
        outputs, hidden, cell = self.decoder(decoder_inputs, hidden, cell)
        return outputs

def split_data(features, target, input_steps, output_steps, train_ratio, seed):
    X, y, decoder_inputs = [], [], []
    total_size = input_steps + output_steps
    for i in range(len(features) - total_size + 1):
        X.append(features[i:i + input_steps])
        y.append(target[i + input_steps:i + total_size])
        decoder_inputs.append(target[i + input_steps - 1:i + input_steps + output_steps - 1])
    
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = train_test_split(
        X, y, decoder_inputs, train_size=train_ratio, random_state=seed
    )
    return X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test


Bitcoin = pd.read_csv('data/coin_Bitcoin.csv')
data = Bitcoin.iloc[:, 4:]
features = data[['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']]
features = MinMaxScaler().fit_transform(features)  # normalize input
target = data['Close']
target_reshaped = np.array(target).reshape(-1, 1)  # normalize output
scaler = MinMaxScaler(feature_range=(0, 1))
target = scaler.fit_transform(target_reshaped).flatten()

# Define our parameters
input_steps = 6
output_steps = 5
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_steps = [[] for _ in range(output_steps)]
mae_steps = [[] for _ in range(output_steps)]
mape_steps = [[] for _ in range(output_steps)]

train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_steps)]
train_mae_steps = [[] for _ in range(output_steps)]
train_mape_steps = [[] for _ in range(output_steps)]

for exp in range(num_experiments):
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = split_data(
        features, target, input_steps, output_steps, train_ratio, seed
    )
    train_dataset = Time_Series_Dataset(X_train, decoder_inputs_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, decoder_inputs_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # Training loop
    input_size = features.shape[1]  # number of features
    hidden_size = 100
    output_size = 1  # predicting 1 value per time step
    num_layers = 2  # Two LSTM networks with a time-distributed layer
    
    model = EncoderDecoderLSTM(input_size, hidden_size, output_size, num_layers)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 100 # default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
        # if epoch == 1 or epoch % 10 == 0:
            # print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.6f}')

    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in test_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            
            y_pred.append(outputs.numpy())
            y_test.append(targets.numpy())
    
    # Convert lists to numpy arrays and remove last dimension
    y_pred = np.concatenate(y_pred, axis=0).squeeze(-1)
    y_test = np.concatenate(y_test, axis=0).squeeze(-1)

    mse = mean_squared_error(y_test, y_pred)
    rmse.append(np.sqrt(mse))
    
    for step in range(output_steps):
        mse_step = mean_squared_error(y_pred[:, step], y_test[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))

    # Inverse Transform
    predicted_values = scaler.inverse_transform(y_pred)
    actual_values = scaler.inverse_transform(y_test)
    
    mae.append(mean_absolute_error(actual_values, predicted_values))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_values))
    
    actual_values_steps = list(zip(*actual_values))
    predicted_values_steps = list(zip(*predicted_values))
    
    for step in range(output_steps):
        mae_steps[step].append(mean_absolute_error(actual_values_steps[step], predicted_values_steps[step]))
        mape_steps[step].append(mean_absolute_percentage_error(actual_values_steps[step], predicted_values_steps[step]))

    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.numpy())

    # Convert lists to numpy arrays and remove last dimension
    y_train_pred = np.concatenate(y_train_pred, axis=0).squeeze(-1)
    y_train_actual = np.concatenate(y_train_actual, axis=0).squeeze(-1)

    train_mse = mean_squared_error(y_train_actual, y_train_pred)
    train_rmse.append(np.sqrt(train_mse))
    
    for step in range(output_steps):
        train_mse_step = mean_squared_error(y_train_pred[:, step], y_train_actual[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform
    train_predicted_values = scaler.inverse_transform(y_train_pred)
    train_actual_values = scaler.inverse_transform(y_train_actual)
    
    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_values))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_values))
    
    train_actual_values_steps = list(zip(*train_actual_values))
    train_predicted_values_steps = list(zip(*train_predicted_values))
    
    for step in range(output_steps):
        train_mae_steps[step].append(mean_absolute_error(train_actual_values_steps[step], train_predicted_values_steps[step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_actual_values_steps[step], train_predicted_values_steps[step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1

print(f"Bitcoin ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse):.4f} {pm} {np.std(rmse):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%"
    )


Experiment 1/30 done
Experiment 2/30 done
Experiment 3/30 done
Experiment 4/30 done
Experiment 5/30 done
Experiment 6/30 done
Experiment 7/30 done
Experiment 8/30 done
Experiment 9/30 done
Experiment 10/30 done
Experiment 11/30 done
Experiment 12/30 done
Experiment 13/30 done
Experiment 14/30 done
Experiment 15/30 done
Experiment 16/30 done
Experiment 17/30 done
Experiment 18/30 done
Experiment 19/30 done
Experiment 20/30 done
Experiment 21/30 done
Experiment 22/30 done
Experiment 23/30 done
Experiment 24/30 done
Experiment 25/30 done
Experiment 26/30 done
Experiment 27/30 done
Experiment 28/30 done
Experiment 29/30 done
Experiment 30/30 done
Bitcoin ED-LSTM Regression: After 30 experimental runs, here are the results:


NameError: name 'pm' is not defined

In [2]:

pm = "\u00B1"
print(f"Bitcoin ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse):.4f} {pm} {np.std(rmse):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%"
    )


Bitcoin ED-LSTM Regression: After 30 experimental runs, here are the results:
Across 5 predictive time steps on the test dataset, Avg RMSE: 0.0114 ± 0.0010, Avg MAE: 332.12 ± 48.22, Avg MAPE: 18.285% ± 9.798%
At time step 1 on the test dataset, Avg RMSE: 0.0101 ± 0.0013, Avg MAE: 303.52 ± 57.27, Avg MAPE: 20.011% ± 11.892%
At time step 2 on the test dataset, Avg RMSE: 0.0112 ± 0.0012, Avg MAE: 316.62 ± 45.64, Avg MAPE: 15.802% ± 11.382%
At time step 3 on the test dataset, Avg RMSE: 0.0118 ± 0.0013, Avg MAE: 334.66 ± 49.71, Avg MAPE: 16.077% ± 11.018%
At time step 4 on the test dataset, Avg RMSE: 0.0119 ± 0.0014, Avg MAE: 348.67 ± 65.21, Avg MAPE: 18.566% ± 11.647%
At time step 5 on the test dataset, Avg RMSE: 0.0117 ± 0.0018, Avg MAE: 357.13 ± 70.97, Avg MAPE: 20.969% ± 13.408%
Across 5 predictive time steps on the train dataset, Avg RMSE: 0.0115 ± 0.0009, Avg MAE: 336.46 ± 47.84, Avg MAPE: 18.129% ± 9.562%
At time step 1 on the train dataset, Avg RMSE: 0.0103 ± 0.0005, Avg MAE: 307.06

In [3]:
quantile_dfs = []
for i in range(pred_values.shape[2]):
    quantile_df = pd.DataFrame(
        pred_values[:, :, i], 
        columns=[f'Quantile_{i+1}_Timestep_{j+1}' for j in range(pred_values.shape[1])]
    )
    quantile_dfs.append(quantile_df)
predicted_df = pd.concat(quantile_dfs, axis=1)

predicted_df.to_csv('bitcoin_multi_edlstm_quantile_results.csv')
pd.DataFrame(actual_values).to_csv('bitcoin_multi_edlstm_quantile_actual.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('bitcoin_multi_edlstm_quantile_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('bitcoin_multi_edlstm_quantile_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('bitcoin_multi_edlstm_quantile_mape.csv')

NameError: name 'pred_values' is not defined